In [1]:

# coding: utf-8

# In[1]:


## First try - freeze weights except that of the last layer...train the last layer.

from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
plt.ion()   # interactive mode

import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
# In[2]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

#~/PycharmProjects/SML/resnet_data
data_dir = '/home/zhen/PycharmProjects/SML/resnet_data'
# data_dir = '~/PycharmProjects/SML/resnet_data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=80,
                                             shuffle=True, num_workers=8)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

use_gpu = torch.cuda.is_available()



In [3]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=12): #initially was 25
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for data in dataloaders[phase]:
                # get the inputs
                inputs, labels = data

                # wrap them in Variable
                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data[0] * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model



In [7]:
#model_conv = torchvision.models.resnet50(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False # False for frozen weights..
    # if given false...batch size = 65
    # if true...batch size = 15

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 14951)

if use_gpu:
    model_conv = model_conv.cuda()
#crossentropyloss
criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opoosed to before.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)
# Adam optimization 
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=12) # initially was 25

torch.save(model_conv.state_dict(), 'recognition_resnet50_nofreeze_only change in final layer_modded.pt')

Epoch 0/11
----------
train Loss: 6.6388 Acc: 0.1896
val Loss: 5.6580 Acc: 0.2736

Epoch 1/11
----------
train Loss: 5.6642 Acc: 0.2678
val Loss: 5.0135 Acc: 0.3407

Epoch 2/11
----------
train Loss: 5.1970 Acc: 0.3140
val Loss: 4.6023 Acc: 0.3843

Epoch 3/11
----------
train Loss: 4.8738 Acc: 0.3472
val Loss: 4.2939 Acc: 0.4168

Epoch 4/11
----------
train Loss: 4.6303 Acc: 0.3721
val Loss: 4.0583 Acc: 0.4400

Epoch 5/11
----------
train Loss: 4.4320 Acc: 0.3912
val Loss: 3.8663 Acc: 0.4608

Epoch 6/11
----------
train Loss: 4.2690 Acc: 0.4078
val Loss: 3.7207 Acc: 0.4763

Epoch 7/11
----------
train Loss: 4.1763 Acc: 0.4168
val Loss: 3.6839 Acc: 0.4810

Epoch 8/11
----------
train Loss: 4.1579 Acc: 0.4192
val Loss: 3.6785 Acc: 0.4814

Epoch 9/11
----------
train Loss: 4.1460 Acc: 0.4204
val Loss: 3.6605 Acc: 0.4827

Epoch 10/11
----------
train Loss: 4.1320 Acc: 0.4218
val Loss: 3.6517 Acc: 0.4845

Epoch 11/11
----------
train Loss: 4.1196 Acc: 0.4231
val Loss: 3.6282 Acc: 0.4858

Tr

In [5]:
'''
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated
'''

# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])


NameError: name 'imshow' is not defined

In [30]:
class FaceLandmarksDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.landmarks_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,
                                self.landmarks_frame.iloc[idx, 0])
        if(os.path.isfile(img_name+".jpg")):
            image = Image.open(img_name+".jpg")
            if self.transform:
                image = self.transform(image)
        else:
            image = None # Image.new('RGB', (256,256), (255, 255, 255))
        filename = img_name

        return image, filename

    
data_transforms_test = {
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

transformed_dataset = {x: FaceLandmarksDataset(csv_file='test_new.csv',
                                           root_dir='/home/zhen/PycharmProjects/SML/input/test/test-sub/',
                                           transform=data_transforms_test[x])
                      for x in ['test']}

dataloaders_test = {x: torch.utils.data.DataLoader(transformed_dataset[x], batch_size=100,
                                             shuffle=False, num_workers=8)
                   for x in ['test']}
use_gpu = torch.cuda.is_available()

In [31]:
sys.stdout=open("test.txt",'w')
import sys
use_gpu = torch.cuda.is_available()
import torch.utils.data
def test_model(model):
    model.eval()
    for i, data in enumerate(dataloaders_test['test']):
    #for data in dataloaders_test['test']: 
        #print(data)
        inputs,labels = data
        print(labels)
        print(i)
        #if use_gpu:
        inputs = Variable(inputs.cuda())
        #else:
        #inputs = Variable(inputs)
        #outputs = model(inputs)
        values, preds = torch.max((model(inputs)).data, 1)
        preds_cp = (Variable(preds).data).cpu().numpy()
        values_cp = (Variable(values).data).cpu().numpy()
        for j in range(len(preds_cp)):
            print(preds_cp[j], " ",values_cp[j])

In [16]:
dataloaders_test

{'test': <torch.utils.data.dataloader.DataLoader at 0x7f995a567898>}

In [4]:
model_conv = torchvision.models.resnet50(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 14951)

In [5]:
model_conv.load_state_dict(torch.load('recognition_resnet50_nofreeze_only change in final layer.pt'))

In [84]:
dataloaders_test['test']

In [32]:

if use_gpu:
    model_conv = model_conv.cuda()
    
test_model(model_conv)

Exception ignored in: <bound method DataLoaderIter.__del__ of <torch.utils.data.dataloader.DataLoaderIter object at 0x7f995a5d9668>>
Traceback (most recent call last):
  File "/home/zhen/anaconda3/envs/sml/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 333, in __del__
    self._shutdown_workers()
  File "/home/zhen/anaconda3/envs/sml/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 319, in _shutdown_workers
    self.data_queue.get()
  File "/home/zhen/anaconda3/envs/sml/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/home/zhen/anaconda3/envs/sml/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 70, in rebuild_storage_fd
    fd = df.detach()
  File "/home/zhen/anaconda3/envs/sml/lib/python3.6/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/home/zhen/anaconda3/envs/sml/lib/python3.6/multiprocessing/

AttributeError: Traceback (most recent call last):
  File "/home/zhen/anaconda3/envs/sml/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 55, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/zhen/anaconda3/envs/sml/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 135, in default_collate
    return [default_collate(samples) for samples in transposed]
  File "/home/zhen/anaconda3/envs/sml/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 135, in <listcomp>
    return [default_collate(samples) for samples in transposed]
  File "/home/zhen/anaconda3/envs/sml/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 109, in default_collate
    numel = sum([x.numel() for x in batch])
  File "/home/zhen/anaconda3/envs/sml/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 109, in <listcomp>
    numel = sum([x.numel() for x in batch])
AttributeError: 'NoneType' object has no attr